<a href="https://colab.research.google.com/github/JotaBlanco/QuixStreamsNotebooks/blob/main/Workshops/Scandio/Quix_Streams_PROCESS_CHAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quix Streams
Just use pip install to download the Quix Streams library.

[Quix Streams](https://github.com/quixio/quix-streams) is an open source Python library for processing streaming data. It’s aimed at people who work with time-series data streams — from developers and ML engineers to data scientists and data engineers.

In [1]:
! pip install quixstreams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 36.9 MB/s eta 0:00:00


# Import the libraries
We will be using mainly pandas, quix, matplotlib and seaborn.

In [2]:
import pandas as pd
import quixstreams as qx

# 1 - Create client
Let's start by creating a Quix client that we'll use to publish and subscribe to Kafka topics.

In [3]:
# Initiating Quix managed token, but it could be your own kafka
token = 'sdk-296f2b9decff4770a525ff7d8855a78d'
client = qx.QuixStreamingClient(token)
client

# 2 - Clients
Create producer and consumer clients

In [4]:
topic_name = "chat-messages-enriched"
topic_producer = client.get_topic_producer(topic_name)
topic_producer

In [14]:
stream_id = "scandio"
stream_out = topic_producer.get_or_create_stream(stream_id)
stream_out

In [6]:
topic_name = "chat-messages"
topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer

# 3 - Listen to some data
Let's listen to some data

In [7]:
df= pd.DataFrame()

def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream: qx.StreamConsumer, df_i: pd.DataFrame):
  global df
  df = df.append(df_i)
  print("Data from stream " + stream.stream_id)
  display(df_i)

topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

Data from stream scandio


<ipython-input-7-ca5b122323bc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_i)


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email
0,1686654590865000000,hi,scandio,Customer,Javier PC,,


Data from stream scandio


<ipython-input-7-ca5b122323bc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_i)


...

<ipython-input-7-ca5b122323bc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_i)


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email
0,1686654601695000000,sad message,scandio,Customer,Javier PC,,


Data from stream scandio


<ipython-input-7-ca5b122323bc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_i)


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email
0,1686654604396000000,let's do one more,scandio,Customer,Javier PC,,


In [8]:
df

,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email
0,1686654590865000000,hi,scandio,Customer,Javier PC,,
0,1686654594915000000,this is a message,scandio,Customer,Javier PC,,
0,1686654597284000000,hello,scandio,Customer,Javier PC,,
0,1686654599354000000,happy days,scandio,Customer,Javier PC,,
0,1686654601695000000,sad message,scandio,Customer,Javier PC,,
0,1686654604396000000,let's do one more,scandio,Customer,Javier PC,,


# 4 - Process data with Hugging Face

In [9]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00


In [10]:
from transformers import pipeline

In [11]:
pipeline_model = pipeline(model='siebert/sentiment-roberta-large-english')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [12]:
pd.DataFrame(pipeline_model(["This is analysing text", "Two messages"]))

,label,score
0,POSITIVE,0.985885
1,NEGATIVE,0.996774


In [13]:
prediction = pipeline_model("This is analysing text")[0]
prediction["label"], prediction["score"]

('POSITIVE', 0.9858850240707397)

## 4.1 - Processing without state

In [15]:
topic_producer = client.get_topic_producer("chat-messages-enriched")
stream_out = topic_producer.get_or_create_stream(stream_id)

def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream_in: qx.StreamConsumer, df: pd.DataFrame):

  # Add predictions
  df_prediction = pd.DataFrame(pipeline_model(list(df["chat-message"])))
  df = pd.concat([df, df_prediction], axis=1)

  # Sentiment column
  df["sentiment"] = df["score"]
  filter_negative = df["label"] == "NEGATIVE"
  df.loc[filter_negative, "sentiment"] = -df.loc[filter_negative, "score"]

  # Average
  #df["average_sentiment"] = df["sentiment"]
  display(df)
  stream_out.timeseries.publish(df)

topic_consumer = client.get_topic_consumer("chat-messages")
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment
0,1686655084228000000,happy days,scandio,Customer,Javier PC,,,POSITIVE,0.998586,0.998586


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment
0,1686655093050000000,sad message,scandio,Customer,Javier PC,,,NEGATIVE,0.998321,-0.998321


## 4.2 - Processing with state

In [16]:
topic_producer = client.get_topic_producer("chat-messages-enriched")
stream_out = topic_producer.get_or_create_stream(stream_id)

last_X_sent = []

def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream_in: qx.StreamConsumer, df: pd.DataFrame):
  global last_X_sent

  # Add predictions
  df_prediction = pd.DataFrame(pipeline_model(list(df["chat-message"])))
  df = pd.concat([df, df_prediction], axis=1)

  # Sentiment column
  df["sentiment"] = df["score"]
  filter_negative = df["label"] == "NEGATIVE"
  df.loc[filter_negative, "sentiment"] = -df.loc[filter_negative, "score"]

  # Average
  last_X_sent = last_X_sent + list(df["sentiment"])
  last_X_sent = last_X_sent[-5:]
  df["average_sentiment"] = sum(last_X_sent)/len(last_X_sent)
  display(df)
  stream_out.timeseries.publish(df)

topic_consumer = client.get_topic_consumer("chat-messages")
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655167721000000,hi,scandio,Customer,Javier PC,,,POSITIVE,0.994815,0.994815,0.994815


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655174112000000,happy=,scandio,Customer,Javier PC,,,POSITIVE,0.998671,0.998671,0.996743


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655175131000000,sad,scandio,Customer,Javier PC,,,NEGATIVE,0.997557,-0.997557,0.331976


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655178553000000,sadder,scandio,Customer,Javier PC,,,NEGATIVE,0.997871,-0.997871,-0.000485


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655186144000000,bollocks,scandio,Customer,Javier PC,,,NEGATIVE,0.998601,-0.998601,-0.200109


...

,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655197153000000,happy birthday,scandio,Customer,Javier PC,,,POSITIVE,0.998874,0.998874,-0.572345


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655203122000000,happy days,scandio,Customer,Javier PC,,,POSITIVE,0.998586,0.998586,-0.173117


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655205642000000,happy,scandio,Customer,Javier PC,,,POSITIVE,0.998799,0.998799,0.226217


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655208523000000,good feeling,scandio,Customer,Javier PC,,,POSITIVE,0.998566,0.998566,0.625651


,timestamp,chat-message,TAG__room,TAG__role,TAG__name,TAG__phone,TAG__email,label,score,sentiment,average_sentiment
0,1686655210591000000,felling great,scandio,Customer,Javier PC,,,POSITIVE,0.9986,0.9986,0.998685
